In [1]:
! pip install peft bitsandbytes accelerate

In [2]:
import os
from typing import List, Dict, Any

import torch
import torch.nn as nn
from torch.nn.utils.rnn import pad_sequence
import torch.optim as optim
from transformers import AutoModelForCausalLM, AutoTokenizer, Trainer, TrainingArguments, BitsAndBytesConfig, AutoConfig, set_seed
from torch.utils.data import Dataset
from datasets import load_dataset
from peft import PeftModel, get_peft_model, LoraConfig, prepare_model_for_kbit_training

set_seed(12, True)

os.environ["WANDB_DISABLED"] = "true"
os.environ["CUBLAS_WORKSPACE_CONFIG"] = ":16:8"

c:\Users\Dmitry\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Gradient Accumulation

Давайте реализуем собственную аккумуляцию градиентов.
Ниже описано обучение обычного линейного слоя. Клеткой ниже этот код скопирован, там необходимо написать аккумуляцию ргадиентов.

In [3]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
input_size = 512
output_size = 256
batch_size = 64
gradient_accumulation_steps = 4



model = nn.Linear(input_size, output_size).to(device)
optimizer = optim.SGD(model.parameters(), lr=0.001)

x = torch.randn(batch_size, input_size).to(device)
y = torch.randn(batch_size, output_size).to(device)
loss_fn = nn.MSELoss()
for i in range(1000):
    optimizer.zero_grad()
    output = model(x)
    loss = loss_fn(output, y)
    loss.backward()
    optimizer.step()

print(loss.item())

1.1878371238708496


Число шагов в аккумуляции определяется параметром gradient_accumulation_steps - это число шагов, которое мы хотим сделать перед оптимизацией.
Вам нужно поправить цикл обучения следующим образом:
1. Разбить текущий батч на gradient_accumulation_steps частей
2. Пройтись по каждому подбатчу (микробатчу), посчитать на нем функцию потерь, посчитать градиенты. Подумайте, нужно ли на что-либо делить или умножать функцию потерь, чтобы сохранился тот же масштаб обучения?
3. После прохождения всех микробатчей нужно сделать шаг оптимизации

In [4]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
input_size = 512
output_size = 256
batch_size = 64
gradient_accumulation_steps = 4

micro_batch_size = batch_size // gradient_accumulation_steps

model = nn.Linear(input_size, output_size).to(device)
optimizer = optim.SGD(model.parameters(), lr=0.001)

x = torch.randn(batch_size, input_size).to(device)
y = torch.randn(batch_size, output_size).to(device)
loss_fn = nn.MSELoss()
for i in range(1000):
    optimizer.zero_grad()
    for j in range(gradient_accumulation_steps):
        x_mb = x[j*micro_batch_size:(j+1)*micro_batch_size]
        y_mb = y[j*micro_batch_size:(j+1)*micro_batch_size]
        output = model(x_mb)
        loss = loss_fn(output, y_mb) / gradient_accumulation_steps
        loss.backward()
    optimizer.step()

print(loss.item() * gradient_accumulation_steps)

1.1768290996551514


# QLORA
Необходимо использовать аккумуляцию градиентов, чекпоинтинг активаций и обучение qlora.

In [5]:
model_name = "NousResearch/Llama-2-7b-hf"
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [6]:
imdb = load_dataset("imdb")

Наша задача научиться генерировать класс текста posive или negative, чтобы сэкономить на fewshot промпте.

Давайте напишем collate_fn, которая собирает сэмпл следующим образом:

если текст имеет метку 1
`{text} ||| posive eos`
или
`{text} ||| negatve eos`
если текст имеет метку 0. (в качестве eos можно использовать tokenizer.eos_token_id)

Символы ||| нужны нам, чтобы разделить входной текст и метку, иначе модель может не понять, что нужно генерировать метку и продолжит генерировать текст. Таким образом мы научим модель после ||| генерировать положительный или отрицательнй отзыв стоит до этого.


Возвращать нужно словарь из 3х элементов:
1. input_ids - LongTensor токенов. В качестве паддинга нужно использовать tokenizer.eos_token_id.
2. attention_mask - LongTensor той же размерности, что и input_ids. 0 там, где стоят паддинги, 1 в остальных позициях
3. labels - метки, которые мы предсказыаем. Должен быть равен -100 на всех позициях, кроме позиций, которые соответствуют метке и eos символу.
Например
```python
tokenizer.encode("some text ||| positive </s>") # [1, 777, 1426, 3830, 29989, 6374, 2]
labels = [-100, -100, -100, -100, -100, 6374, 2]
```

Т.е. метки должны быть -100, кроме позиций, соответствующих предсказываемым токенам.

In [12]:
class_mapping = {0: "negative", 1: "positive"}

def collate_fn(batch: List[Dict[str, Any]]):    
    sep_ids = tokenizer.encode(" ||| ", add_special_tokens=False)
    pos_ids = tokenizer.encode(class_mapping[1], add_special_tokens=False) + [tokenizer.eos_token_id]
    neg_ids = tokenizer.encode(class_mapping[0], add_special_tokens=False) + [tokenizer.eos_token_id]
    target_length = max(len(pos_ids), len(neg_ids))
    keep = 4096 - len(sep_ids) - target_length

    raw_texts = [sample["text"] for sample in batch]

    tokenized = tokenizer(
        raw_texts,
        add_special_tokens=False,
        padding=False,
        truncation=True,
        max_length=keep
    )

    inputs = []
    labels = []
    lengths = []
    for i, text_ids in enumerate(tokenized["input_ids"]):
        label_ids = pos_ids if batch[i]["label"] == 1 else neg_ids
        input_ids = torch.tensor(text_ids + sep_ids + label_ids, dtype=torch.long)
        
        prefix_len = len(text_ids) + len(sep_ids)
        label = torch.tensor([-100] * prefix_len + label_ids, dtype=torch.long)

        inputs.append(input_ids)
        labels.append(label)
        lengths.append(input_ids.size(0))
    
    inputs = pad_sequence(inputs, padding_value=tokenizer.eos_token_id, batch_first=True)
    labels = pad_sequence(labels, padding_value=-100, batch_first=True)

    attention_mask = torch.zeros_like(inputs, dtype=torch.long)
    for i, L in enumerate(lengths):
        attention_mask[i, :L] = 1

    return {
        "input_ids": inputs,
        "attention_mask": attention_mask,
        "labels": labels
    }

res = collate_fn([imdb["train"][0], imdb["train"][12505], imdb["train"][2]])

assert tokenizer.decode(res["input_ids"][res["labels"] != -100]) == "negative</s> positive</s> negative</s>"

Далее нам нужно создать модель в nf4, т.е. 4-битной квантизации. Конфиг уже написан, нужно лишь подать его в модель. После этого нужно:
1. Создать конфиг адаптера LoraConfig (используйте r=8 или r=4, если будет OOM) и создать модель
2. Создать модель с адаптером с помощью PeftModel и LoraConfig
3. Чтобы обучение шло только по lora частям, нужно пройтись по всем параметрам модели с помощью model.named_parameters() и проставить у параметров, соответствующих lora атрибут requires_grad = True, а у всех остальных False

In [8]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True, # храним веса в 4-битном виде (коды 0..15)
    bnb_4bit_quant_type="nf4", # определяет схему квантизации, по которой выбирается один из 16 уровней
    bnb_4bit_compute_dtype=torch.bfloat16, # тип, в котором выполняются вычисления. bfloat - больше диапазон, меньше точность
    bnb_4bit_use_double_quant=True, # двойная квантизация - квантуются не только веса, но и статистики для декодирования
#    bnb_4bit_quant_storage=torch.bfloat16, тип хранения кодов, тут странное значение
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    torch_dtype=torch.bfloat16,
)

model.config.use_cache = False # не используем KV-cache во время обучения

model = prepare_model_for_kbit_training(model) # стабилизирует LayerNorm, активирует нужные флаги, сочетается с gradient checkpointing

model.gradient_checkpointing_enable()

peft_config = LoraConfig(
    r=8,
    lora_alpha=16,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=[
        "q_proj",
        "v_proj"
    ]
)

model = get_peft_model(model, peft_config)

for name, p in model.named_parameters():
    if "lora_" in name:
        p.requires_grad = True
    else:
        p.requires_grad = False

`torch_dtype` is deprecated! Use `dtype` instead!
Loading checkpoint shards: 100%|██████████| 2/2 [00:29<00:00, 14.55s/it]


Осталось самое важное, аргументы обучения. Обязательно заполните следующие параметры:

1. Батч сайз и число шагов аккумуляции выставьте так, чтобы эффективный батч сайз был 16
2. Включите чекпоинтинг активаций

In [9]:
args = TrainingArguments(
    per_device_train_batch_size=2,
    gradient_accumulation_steps=8,
    gradient_checkpointing=True,
    bf16=True,
    max_steps=100,
    learning_rate=2e-4,
    logging_steps=10,
    report_to=None,
    remove_unused_columns=False
)

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=imdb["train"],
    tokenizer=tokenizer,
    data_collator=collate_fn,
)
trainer.train()

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
C:\Users\Dmitry\AppData\Local\Temp\ipykernel_4836\4011455066.py:13: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'pad_token_id': 0}.


Step,Training Loss
10,5.053800
20,0.117500
30,0.075800
40,0.114200
50,0.073500
60,0.065900
70,0.067100
80,0.060800
90,0.066500
100,0.052500


TrainOutput(global_step=100, training_loss=0.5747581481933594, metrics={'train_runtime': 3566.7589, 'train_samples_per_second': 0.449, 'train_steps_per_second': 0.028, 'total_flos': 3.08067253997568e+16, 'train_loss': 0.5747581481933594, 'epoch': 0.064})

Давайте протестируем, что модель что-то выучила

In [18]:
input_text = imdb["test"][3]["text"] + " ||| "
label = imdb["test"][3]["label"]
x = tokenizer(input_text, return_tensors="pt")
for k, v in x.items():
    x[k] = v.cuda()

print(class_mapping[label])
g = model.generate(**x, max_new_tokens=1, do_sample=False)
print(tokenizer.decode(g[0].tolist()))

negative
<s> STAR RATING: ***** Saturday Night **** Friday Night *** Friday Morning ** Sunday Night * Monday Morning <br /><br />Former New Orleans homicide cop Jack Robideaux (Jean Claude Van Damme) is re-assigned to Columbus, a small but violent town in Mexico to help the police there with their efforts to stop a major heroin smuggling operation into their town. The culprits turn out to be ex-military, lead by former commander Benjamin Meyers (Stephen Lord, otherwise known as Jase from East Enders) who is using a special method he learned in Afghanistan to fight off his opponents. But Jack has a more personal reason for taking him down, that draws the two men into an explosive final showdown where only one will walk away alive.<br /><br />After Until Death, Van Damme appeared to be on a high, showing he could make the best straight to video films in the action market. While that was a far more drama oriented film, with The Shepherd he has returned to the high-kicking, no brainer acti